In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from datetime import timedelta
import itertools
import calendar
import random
import pickle
import numba
from numba import njit

# Output Locations:

###### Storage location for statistics from generated random networks

In [2]:
RANDSTATS_LOC = "/Volumes/G-DRIVE/codetest/statsrandomnets_exprob/"

###### Storage location for ML functions

In [3]:
MLF_LOC = "/Volumes/G-DRIVE/codetest/likelihoodfns_exprob/"

# Name the combinations of kappa and Np

**Note that in the paper, we use:**
1. The ranges for overall activity, $\kappa$ and population size $N_\mathrm{p}$:
    - $\kappa$ = $[0,4]$ _incremented by 0.025_
    - $N_\mathrm{p}$ = $[2,230]$ _incremented by 1_
2. All combinations of the parameters are in the dictionary **param_combs_dict**
3. Generating the likelihood functions given these ranges will take considerable amount of time. Hence, for demonstration, we we use a smaller range of values for both parameters.

In [4]:
kappas = np.arange(0.1,0.2,0.025)
Nps = np.arange(5,10,1)

#The parameter combinations are specific to the range of kappas and Nps, hence user can update as needed.
param_combs = [] 
for k in kappas:
    for pop in Nps:
        param_combs.append((k,pop))
        
#each parameter combination is named and used as dictionary keys.
param_keys = ["p"+str(i) for i in range(len(param_combs)+1)]

param_combs_dict = dict(zip(param_keys,param_combs))

# Two-part function for generating the synthetic networks given combination of $\kappa$ and $N_\mathrm{p}$

### Part 1:
###### _probs_ function outputs:
    - Adjmat: (i.e., adjacency matrix) initialize adj mat with the total number of nodes in the system = Np
    - upper_tridex: (i.e., upper triangle) since the network is undirected, we focus on the upper triangle
    - probmatrix: the computed probability of a connection between i and j
    - comp_probab: (i.e.,comparison prob. matrix) randomly drawn value to compare the prob. (of a connection) value 

In [5]:
@njit
def probs(k,Np):
    """
    Inputs:
    Np: Network population size
    k: For kappa which is overall activity level
    -------------------------
    Outputs:
    Adjmat:initialized Np x Np adjacency matrix
    upper_tridex: upper triangle of the adjacency matrix 
    probmatrix: probability matrix with each entry rep. the computed prob. of two nodes i and j connecting
    comp_probab: randomly drawn values against which, the respective entries of "probmatrix" are compared in function
    'populate_uptri'. 
    
    Probability Equation: p_ij = 1-e^(-k*a_i*a_j); hence "actmat" corresponds to the matrix obtained by a_i*a_j.
    This probability equation can be changed to any other equation of choice.
    """

    Adjmat = np.zeros((Np, Np)) #initialize adjacency matrix
    upper_tridex = np.triu_indices(Np,1) #index for the edges in the upper tri one diag above main
    triup_size = upper_tridex[0].size    #num elems in this upper_tridex1
    
    nodeact = np.array([np.random.random() for i in range(Np)]) #assign activity level from uniform dist [0,1]
    actmat = np.outer(nodeact,nodeact)                          #activity matrix for aiaj
    np.fill_diagonal(actmat,0)
    #The proability equation. UPDATE AS DESIRED 
    probmatrix = 1-np.exp((actmat*-k))                       
    comp_probab = np.random.rand(triup_size)
    
    return Adjmat,upper_tridex,probmatrix,comp_probab

### Part 2:
###### _populate_uptri_ function output:
    - popuptriangle: the upper triangle of Adjmat
From this upper triangle, we compute the number of edges (M) and number of active nodes (N). 

In [6]:
@njit
def populate_uptri(Adjmat):
    """
    Takes, as input, the final adjacency matrix (i.e., edges id by 1, 0 otherwise) and outputs the upper triangle. 
    -------------------------
    Inputs:
    Adjmat: initialized Np x Np adjacency matrix
    -------------------------
    Outputs:
    popuptriangle: the upper triangle of Adjmat
    """
    Adjmat = Adjmat + Adjmat.T - np.diag(np.diag(Adjmat))                   
    popuptriangle = np.where(np.triu(Adjmat,1)>0) 
    return popuptriangle
    

## Compute the statistics (total active nodes N and total edges M) of each synthetic network given a parameter combination
- Here, total reps = 10,000 therefore each parameter combination in _param_combs_dict_ has 10,000 synthetic networks each with an N and M.

In [7]:
count = 0 #To track progress
reps = 10000 # number of generated networks per parameter combination
for combids,parvals in param_combs_dict.items():
    count+=1
    Nt_Mt = {}
    for s in range(reps):
        Adj,upper_tridex1,probmat,comp_prob = probs(parvals[0],parvals[1])

        triup_edges = np.where(probmat[upper_tridex1]>=comp_prob,1,0) #compared calc prob (probmat) vs. rand val (comp_prob)

        Adj[upper_tridex1] = triup_edges                             # where edge exists 1, 0 otherwise
        pop_triup = populate_uptri(Adj)
        Ncount = np.unique(np.array(list(itertools.chain.from_iterable(pop_triup)))).size
        Mcount = len(list(zip(list(pop_triup[0]),list(pop_triup[1]))))
        Nt_Mt[s] = {"N":Ncount,"M":Mcount}
    params_df = pd.DataFrame.from_dict(Nt_Mt,orient = "index")
    params_df["combids"] = [combids]*params_df.shape[0]
    params_df.to_pickle(RANDSTATS_LOC+"param_df"+combids)
    #print progress:
    if (count%round(len(param_combs_dict)/10)==0)&(count>0): print(round(count/len(param_combs_dict)*100,0), "% completed")
    
    

10.0 % completed
20.0 % completed
30.0 % completed
40.0 % completed
50.0 % completed
60.0 % completed
70.0 % completed
80.0 % completed
90.0 % completed
100.0 % completed


## Obtaining the Likelihood functions

1. **_Graphcomb_**: The statistics i.e. total edges M and total nodes N) for each of the 10,000 networks generated for a combinations of the parameters. Format: pandas.Dataframe

2. **_Likefn_df_**: The likelihood function for each parameter combination. Format: pandas.Dataframe


In [8]:
for combids,parvals in param_combs_dict.items():
    #____Statistics for each of the 10,000 nets for a parameter combination
    Graphcomb = pd.read_pickle(RANDSTATS_LOC+"param_df"+combids)
    Graphcomb.reset_index(drop = False, inplace = True)
    Graphcomb.rename(columns = {"index":"runs"},inplace = True)
    
    #____Joint probability distribution:
    Likefn_df = Graphcomb.groupby(["N","M"],as_index = False)["runs"].count()
    Likefn_df.rename(columns = {"runs":"freq"},inplace = True)
    Likefn_df["prob"] = Likefn_df["freq"]/Likefn_df["freq"].sum()
    #__Save the output for this combination
    Likefn_df.to_pickle(MLF_LOC+"Likefn"+combids)

## Sample output:

In [9]:
lkdict = {}
for combids,parvals in param_combs_dict.items():
    #iterate over each combination id and call the saved likelihood function
    df = pd.read_pickle(MLF_LOC+"Likefn"+combids)
    lkdict[combids] = df

In [108]:
#######display the likelihood function for parameter combination "p15".
lkdict["p15"]

,N,M,freq,prob
0,198,902,1,0.0001
1,199,965,1,0.0001
2,200,831,1,0.0001
3,200,854,1,0.0001
4,200,914,1,0.0001
...,...,...,...,...
3953,224,1187,1,0.0001
3954,224,1232,1,0.0001
3955,224,1269,1,0.0001
3956,224,1271,1,0.0001


In [22]:
print("Parameter combination is: ", param_combs_dict["p15"])

Parameter combination is:  (0.175, 5)
